In [1]:
from rich.progress import Progress
from rich import traceback

traceback.install()

<bound method InteractiveShell.excepthook of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7a5ec3f6e910>>

In [2]:
from scapy.all import *

In [3]:
from bitstring import BitArray

In [4]:
pcap_file = "../data/raw/Baccichet/OppoFindX3Neo_A.pcap"
# pcap_file = "../data/raw/Baccichet/iPhone11_B.pcap"

In [5]:
# Load pcap file
packets = rdpcap(pcap_file)

In [6]:
packet = packets[2]

In [7]:
packet

<RadioTap  version=0 pad=0 len=15 present=Flags+Rate+Channel+dBm_AntSignal Flags=FCS Rate=5.5 Mbps ChannelFrequency=2412 ChannelFlags=CCK+2GHz dBm_AntSignal=-50 dBm notdecoded='' |<Dot11FCS  subtype=Probe Request type=Management proto=0 FCfield= ID=0 addr1=ff:ff:ff:ff:ff:ff (RA=DA) addr2=22:e4:72:fb:91:70 (TA=SA) addr3=ff:ff:ff:ff:ff:ff (BSSID/STA) SC=32592 fcs=0xa16bf2bd |<Dot11ProbeReq  |<Dot11Elt  ID=SSID len=12 info='Anto_HotSpot' |<Dot11EltRates  ID=Supported Rates len=4 rates=[1.0 Mbps, 2.0 Mbps, 5.5 Mbps, 11.0 Mbps] |<Dot11EltRates  ID=Extended Supported Rates len=8 rates=[6.0 Mbps, 9.0 Mbps, 12.0 Mbps, 18.0 Mbps, 24.0 Mbps, 36.0 Mbps, 48.0 Mbps, 54.0 Mbps] |<Dot11EltDSSSet  ID=DSSS Set len=1 channel=1 |<Dot11EltHTCapabilities  ID=HT Capabilities len=26 L_SIG_TXOP_Protection=0 Forty_Mhz_Intolerant=0 PSMP=0 DSSS_CCK=0 Max_A_MSDU=3839 o Delayed_BlockAck=0 Rx_STBC=1 Tx_STBC=1 Short_GI_40Mhz=1 Short_GI_20Mhz=1 Green_Field=0 SM_Power_Save=disabled Supported_Channel_Width=20Mhz+40Mhz 

In [8]:
def getMACLayerBits(packet: scapy.layers.dot11.RadioTap) -> BitArray:
    headerLength = packet.getlayer(RadioTap).len
    headerBits = headerLength * 8

    # bit_string = BitArray(bytes=packet_bytes)

    # Convert the packet to bytes
    packet_bytes = bytes(packet)

    # Convert bytes to binary string
    binary_string = "".join(format(byte, "08b") for byte in packet_bytes)

    return binary_string[headerBits:]

In [9]:
packet_bits = getMACLayerBits(packet)
print(packet_bits)

0100000000000000000000000000000011111111111111111111111111111111111111111111111100100010111001000111001011111011100100010111000011111111111111111111111111111111111111111111111101010000011111110000000000001100010000010110111001110100011011110101111101001000011011110111010001010011011100000110111101110100000000010000010000000010000001000000101100010110001100100000100000001100000100100001100000100100001100000100100001100000011011000000001100000001000000010010110100011010111011110000000100010011111111111111111100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000010111111000011001001001011110001100100000011001111111010111111110110001000000011111110101111111101100010000000111101110100000111000000000101000011110010000010000000000000101000000000000111111100001011000000000000000000001010100000100000000001000000000000000000000000000000000000010010000011111111000000110000001000000000

In [10]:
packet_IE = packet_bits[192:]

In [11]:
def readElementID(packet: str) -> str:
    return int(packet[0:8], 2)


def readBinElementID(packet: str) -> str:
    elementID = packet[0:8]
    return elementID


def readBinLength(packet: str) -> str:
    length = packet[8:16]
    return length


def convertBinLength(packet: str) -> int:
    bitLength = int(packet[8:16], 2) * 8
    return int(bitLength)


def readBinField(packet: str) -> str:
    field = packet[16 : 16 + convertBinLength(packet)]
    return field

In [12]:
ELEMENT_IDs = {
    0: "ssid",
    1: "supported rates",
    3: "dsss parameters",
    45: "ht capabilities",
    50: "extended supported rates",
    127: "extended capabilities",
    189: "cgr group address",
    191: "vht capabilities",
    221: "vendor specific tags",
    255: "extended tags",
}

In [13]:
index = 0
packetLength = len(packet_IE)
elements = []

while index < packetLength - 32:
    packet_slice = packet_IE[index:]
    elementID = readBinElementID(packet_slice)
    convertedID = readElementID(packet_slice)
    length = readBinLength(packet_slice)
    field = readBinField(packet_slice)

    elements.append(
        (
            ELEMENT_IDs[convertedID],
            convertBinLength(packet_slice),
            elementID,
            length,
            field,
        )
    )

    index += 16 + convertBinLength(packet_slice)

frame_check_seq = packet_IE[-32:]


elements

[('ssid',
  96,
  '00000000',
  '00001100',
  '010000010110111001110100011011110101111101001000011011110111010001010011011100000110111101110100'),
 ('supported rates',
  32,
  '00000001',
  '00000100',
  '00000010000001000000101100010110'),
 ('extended supported rates',
  64,
  '00110010',
  '00001000',
  '0000110000010010000110000010010000110000010010000110000001101100'),
 ('dsss parameters', 8, '00000011', '00000001', '00000001'),
 ('ht capabilities',
  208,
  '00101101',
  '00011010',
  '1110111100000001000100111111111111111111000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000'),
 ('vht capabilities',
  96,
  '10111111',
  '00001100',
  '100100101111000110010000001100111111101011111111011000100000001111111010111111110110001000000011'),
 ('vendor specific tags',
  56,
  '11011101',
  '00000111',
  '00000000010100001111001000001000000000000010100000000000'),
 ('extende

In [14]:
elements[1][0]

'supported rates'